In [2]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
# from ydata_profiling import ProfileReport
import scipy.stats as stats 
from scipy.stats import mode

# Suppress warnings
import warnings 
warnings.filterwarnings("ignore")

# Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler,RobustScaler
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight

# Model selection and evaluation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score,RandomizedSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, accuracy_score, auc, precision_recall_curve, average_precision_score

# Machine learning models
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
import xgboost as xgb
import lightgbm as lgb

# Visualization of feature importances
from yellowbrick.model_selection import FeatureImportances

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)

In [16]:
df = pd.read_csv('../data/datasets_inter/cleaned.csv')
print('This dataset has %d rows dan %d columns.\n' % df.shape)

df = df.drop(["id", "customer_id", "name", "ssn"], axis=1)
df.head()

This dataset has 100000 rows dan 28 columns.



,month,age,occupation,annual_income,monthly_inhand_salary,total_emi_per_month,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,January,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,26.822620,265,No,80.415300,High_spent_Small_value_payments,312.494080,2
1,February,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,31.944960,266,No,118.280220,Low_spent_Large_value_payments,284.629150,2
2,March,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,28.609352,267,No,81.699524,Low_spent_Medium_value_payments,331.209870,2
3,April,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",5.000000,4,6.270000,4,Good,809.980000,31.377861,268,No,199.458070,Low_spent_Small_value_payments,223.451310,2
4,May,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",6.000000,4,11.270000,4,Good,809.980000,24.797346,269,No,41.420155,High_spent_Medium_value_payments,341.489230,2


In [17]:
eligible_clients = df[df['credit_score'].isin([1, 2])]

In [18]:
df_filtered = df[df['credit_score'].isin([1, 2])]

df_filtered.head()

,month,age,occupation,annual_income,monthly_inhand_salary,total_emi_per_month,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,January,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,26.822620,265,No,80.415300,High_spent_Small_value_payments,312.494080,2
1,February,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,31.944960,266,No,118.280220,Low_spent_Large_value_payments,284.629150,2
2,March,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",3.000000,7,11.270000,4,Good,809.980000,28.609352,267,No,81.699524,Low_spent_Medium_value_payments,331.209870,2
3,April,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",5.000000,4,6.270000,4,Good,809.980000,31.377861,268,No,199.458070,Low_spent_Small_value_payments,223.451310,2
4,May,23,Scientist,19114.120000,1824.843400,49.574947,3,4,3,4,"['Auto Loan', 'Credit-Builder Loan', 'Personal...",6.000000,4,11.270000,4,Good,809.980000,24.797346,269,No,41.420155,High_spent_Medium_value_payments,341.489230,2


In [30]:
def recommend_auto_loan(client):
    """Recommend Auto Loan based on client's profile."""
    high_income_occupations = ['Lawyer', 'Engineer', 'Architect', 'Scientist', 'Accountant', 'Developer']
    
    if ((20 <= client['age'] <= 60 and
         client['annual_income'] > 30000 and
         client['credit_mix'] in ['Good', 'Excellent'] and
         client['payment_behaviour'] in ['High_spent_Large_value_payments', 'High_spent_Medium_value_payments']) or
        ('Auto Loan' not in client['type_of_loan'] and client['occupation'] in high_income_occupations)):
        return True
    return False

In [31]:
def recommend_credit_builder_loan(client):
    """Recommend Credit Builder Loan based on client's profile."""
    low_income_occupations = ['Teacher', 'Musician', 'Writer', 'Journalist']
    
    if ((client['age'] < 35 and
         client['annual_income'] > 10000 and
         client['payment_behaviour'] in ['Low_spent_Small_value_payments', 'Low_spent_Medium_value_payments']) or
        ('Credit-Builder Loan' not in client['type_of_loan']) or
        (client['occupation'] in low_income_occupations)):
        return True
    return False

In [32]:
def recommend_debt_consolidation_loan(client):
    """Recommend Debt Consolidation Loan based on client's profile."""
    if ((client['outstanding_debt'] > 5000 and
         client['credit_utilization_ratio'] > 30 and
         client['num_of_delayed_payment'] > 2) or
        ('Debt Consolidation Loan' not in client['type_of_loan'])):
        return True
    return False

In [33]:
def recommend_home_equity_loan(client):
    """Recommend Home Equity Loan based on client's profile."""
    high_income_occupations = ['Lawyer', 'Engineer', 'Manager', 'Architect', 'Scientist']
    
    if ((client['monthly_balance'] > 2000 and
         client['credit_history_age'] > 5) or
        ('Home Equity Loan' not in client['type_of_loan']) or
        (client['occupation'] in high_income_occupations)):
        return True
    return False

In [34]:
def recommend_mortgage_loan(client):
    """Recommend Mortgage Loan based on client's profile."""
    high_income_occupations = ['Lawyer', 'Engineer', 'Manager', 'Architect', 'Accountant']
    
    if ((client['age'] >= 25 and client['age'] <= 60 and
         client['annual_income'] > 50000) or
        ('Mortgage Loan' not in client['type_of_loan']) or
        (client['occupation'] in high_income_occupations)):
        return True
    return False

In [36]:
def recommend_payday_loan(client):
    """Recommend Payday Loan based on client's profile."""
    
    if ( client['payment_behaviour'] == 'Low_spent_Small_value_payments' or
        'Payday Loan' not in client['type_of_loan']):
        return True
    return False

In [37]:
def recommend_student_loan(client):
    """Recommend Student Loan based on client's profile."""
    
    if ((client['age'] < 30 and client['annual_income'] < 30000) or
        'Student Loan' not in client['type_of_loan']):
        return True
    return False

In [38]:
def loan_recommendation(client):
    """Main function to recommend a loan based on client profile."""
    recommendations = {}

    # Check for loan recommendations based on individual loan functions
    if recommend_auto_loan(client):
        recommendations['Auto Loan'] = True
    if recommend_credit_builder_loan(client):
        recommendations['Credit-Builder Loan'] = True
    if recommend_debt_consolidation_loan(client):
        recommendations['Debt Consolidation Loan'] = True
    if recommend_home_equity_loan(client):
        recommendations['Home Equity Loan'] = True
    if recommend_mortgage_loan(client):
        recommendations['Mortgage Loan'] = True
    if recommend_payday_loan(client):
        recommendations['Payday Loan'] = True
    if recommend_student_loan(client):
        recommendations['Student Loan'] = True

    # Return the loan recommendations
    return recommendations

In [41]:
client = {
    'month': 'January',
    'age': 23,
    'occupation': 'Scientist',
    'annual_income': 19114.12,
    'monthly_inhand_salary': 1824.84,
    'total_emi_per_month': 49.57,
    'num_bank_accounts': 3,
    'num_credit_card': 4,
    'interest_rate': 3,
    'num_of_loan': 4,
    'delay_from_due_date': 7,
    'num_of_delayed_payment': 11.27,
    'changed_credit_limit': 4,
    'credit_inquiries': 0,
    'credit_mix': 'Good',
    'outstanding_debt': 809.98,
    'credit_utilization_ratio': 26.82,
    'credit_history_age': 265,
    'payment_of_min_amount': 'No',
    'amount_invested_monthly': 80.42,
    'payment_behaviour': 'High_spent_Small_value_payments',
    'monthly_balance': 312.49,
    'credit_score': 2,
    'type_of_loan': ['Auto Loan', 'Credit-Builder Loan']
}


In [42]:
recommendations = loan_recommendation(client)
print(recommendations)

{'Debt Consolidation Loan': True, 'Home Equity Loan': True, 'Mortgage Loan': True, 'Payday Loan': True, 'Student Loan': True}
